In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

In [2]:
df = pd.read_csv('DataCleaning.csv', encoding = "ISO-8859-1")

## Five Most Common Journals and How Many?

Well, what do these titles actually look like?

In [3]:
df['Journal title'].value_counts().head(20)

PLoS One                                           92
PLoS ONE                                           62
Journal of Biological Chemistry                    48
Nucleic Acids Research                             21
Proceedings of the National Academy of Sciences    19
Human Molecular Genetics                           18
PLoS Neglected Tropical Diseases                   18
Nature Communications                              17
PLoS Pathogens                                     15
Neuroimage                                         15
PLoS Genetics                                      15
NeuroImage                                         14
Brain                                              14
PLOS ONE                                           14
BMC Public Health                                  14
Movement Disorders                                 13
Journal of Neuroscience                            12
Developmental Cell                                 12
Biochemical Journal         

In [4]:
len(df['Journal title'].value_counts())

984

There are a lot of different kinds of mistakes if we look deeper into the data: extra whitespace, extra punctuation, spelling errors, different capitalization, added s's, multiple languages, etc, etc, etc. Spelling mistakes are going to be a real challenge, but let's see what happens if we strip out all whitespace, punctuation, and capitalize everything.

#### First: Strip whitespace.

In [5]:
df['Journal title'] = df['Journal title'].str.strip()
df['Journal title'].head(5)

0    Psychological Medicine
1         Biomacromolecules
2                J Med Chem
3                J Med Chem
4                J Org Chem
Name: Journal title, dtype: object

Did it do anything?

In [6]:
len(df['Journal title'].value_counts())

951

984 to 951. Making progress!

#### Second: Capitalize. 

In [7]:
df['Journal title'] = df['Journal title'].str.upper()
df['Journal title'].head(5)

0    PSYCHOLOGICAL MEDICINE
1         BIOMACROMOLECULES
2                J MED CHEM
3                J MED CHEM
4                J ORG CHEM
Name: Journal title, dtype: object

In [8]:
len(df['Journal title'].value_counts())

894

That did pretty well!

#### Third: Remove punctuation and spaces.

In [9]:
df['Journal title'] = df['Journal title'].apply(lambda x: ''.join(list(filter(str.isalnum, str(x)))))
df['Journal title'].head(5)

0    PSYCHOLOGICALMEDICINE
1        BIOMACROMOLECULES
2                 JMEDCHEM
3                 JMEDCHEM
4                 JORGCHEM
Name: Journal title, dtype: object

In [10]:
len(df['Journal title'].value_counts())

886

Barely better. So far we've reduced about 100 duplicate journal titles. Let's see how it looks.

In [11]:
df['Journal title'].value_counts().head(10)

PLOSONE                                      200
JOURNALOFBIOLOGICALCHEMISTRY                  53
NEUROIMAGE                                    29
NUCLEICACIDSRESEARCH                          26
PLOSPATHOGENS                                 24
PLOSGENETICS                                  24
PROCEEDINGSOFTHENATIONALACADEMYOFSCIENCES     22
PLOSNEGLECTEDTROPICALDISEASES                 20
NATURECOMMUNICATIONS                          19
HUMANMOLECULARGENETICS                        19
Name: Journal title, dtype: int64

It's a little harder for us to read the journal titles, but a lot easier for the computer to tell which ones are the same. Most of what's left are spelling mistakes from the original data entry, which I'm not sure how to deal with aside from going through by hand.

The five most common journals are PLoS One, Journal of Biological Chemistry, NeuroImage, Nucleic Acids Research, and PLoS Pathogens, with values shown above.

## Next step: Mean, median, and std of the open-access cost per article for each journal

To get cost per article, we'll have to turn the "Cost" column into a number, then do math for each journal. I'll create a second dataframe to avoid issues with counting journals multiple times. Then it should be relatively simple to do statistics on it.

In [12]:
df_price = df['Journal title'].value_counts().to_frame()
df_price.rename(columns={'Journal title': 'Count'}, inplace=True)
df_price['Mean'] = None
df_price['Median'] = None
df_price['Std'] = None
df_price.head(5)

,Count,Mean,Median,Std
PLOSONE,200,None,None,None
JOURNALOFBIOLOGICALCHEMISTRY,53,None,None,None
NEUROIMAGE,29,None,None,None
NUCLEICACIDSRESEARCH,26,None,None,None
PLOSPATHOGENS,24,None,None,None


Turn that 'COST....' column into something we can work with by removing the pound sign and decimal point.

In [13]:
df.rename(columns={'COST (£) charged to Wellcome (inc VAT when charged)': 'Cost in Cents'}, inplace=True)

In [14]:
df['Cost in Cents'] = df['Cost in Cents'].apply(lambda x: int(''.join(list(filter(str.isdigit, str(x))))))

In [15]:
df[df['Cost in Cents']>90000000].head(5)

,PMID/PMCID,Publisher,Journal title,Article title,Cost in Cents
149,PMC3234811,ASBMB,JOURNALOFBIOLOGICALCHEMISTRY,Picomolar nitric oxide signals from central ne...,99999900
227,3708772,BioMed Central,BMCGENOMICS,"Phenotypic, genomic, and transcriptional chara...",99999900
277,PMC3668259,BMC,TRIALS,Community resource centres to improve the heal...,99999900
358,PMC3219211,Cambridge University Press,EXPERTREVIEWSINMOLECULARMEDICINE,Pharmacological targets in the ubiquitin syste...,99999900
404,PMC3533396,Company of Biologists,JOURNALOFCELLSCIENCE,PKA isoforms coordinate mRNA fate during nutri...,99999900


There are a bunch of "99999900" values for price -- these shouldn't be included, or should be reduced to the mean cost of each journal. I'll just mark them as None to remove them from the analysis.

In [16]:
df[df['Cost in Cents']>90000000] = None

In [17]:
for title in df_price.index:
    df_price.loc[title, 'Mean'] = df[df['Journal title']==title]['Cost in Cents'].mean()
    df_price.loc[title, 'Median'] = df[df['Journal title']==title]['Cost in Cents'].median()
    df_price.loc[title, 'Std'] = df[df['Journal title']==title]['Cost in Cents'].std()

In [18]:
df_price.head(5)

,Count,Mean,Median,Std
PLOSONE,200,192999,89407,1.38371e+06
JOURNALOFBIOLOGICALCHEMISTRY,53,142359,130114,41195.4
NEUROIMAGE,29,221517,232643,26665.4
NUCLEICACIDSRESEARCH,26,114900,85200,44294
PLOSPATHOGENS,24,144833,160025,47019.6


Got it! df_price stores all the Mean, Median, and Std information for the price of each journal. More information can be added by including it in that for loop.

## Challenge: Article Price by Subject area?

First off, what are the subject areas in the data set, and how do we know which journals belong to which area? I don't know a lot about how they distinguish subject area, so I'll choose somewhat arbitrarily:

Journal titles including the words:
NEUR, BIO, CHEM, PHYS, NATUR, MOLECUL, HUMAN, MED, CELL, IMMUN, GEN.

This is an incomplete list, I'm sure, and will leave out a lot of the journals. It'll also count some of them twice, which is fine since they may span multiple disciplines.

In [19]:
df_subject = pd.DataFrame(index=['NEUR', 'BIO', 'CHEM', 'PHYS', 'NATUR', 'MOLECUL', 'HUMAN', 'MED', 'CELL', 'IMMUN', 'GEN'])

In [20]:
df_subject['Count'] = None
df_subject['Mean Price'] = None
df_subject.head(5)

,Count,Mean Price
NEUR,None,None
BIO,None,None
CHEM,None,None
PHYS,None,None
NATUR,None,None


In [21]:
for subj in df_subject.index:
    df_subject.loc[subj, 'Count'] = df[df['Journal title'].isin(list(filter(lambda x: subj in str(x), df['Journal title'])))]['Cost in Cents'].count()
    df_subject.loc[subj, 'Mean Price'] = df[df['Journal title'].isin(list(filter(lambda x: subj in str(x), df['Journal title'])))]['Cost in Cents'].mean()

In [22]:
df_subject

,Count,Mean Price
NEUR,192,204948
BIO,329,178561
CHEM,174,161889
PHYS,35,160068
NATUR,20,293776
MOLECUL,90,206824
HUMAN,51,201758
MED,135,171850
CELL,103,247977
IMMUN,51,213125


Nice! Again, you can add other stats by updating the for loop above, or add other categories by redefining the DataFrame.

That's all folks!